In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = ':/root/.mujoco/mujoco200/bin'
# os.environ['LD_PRELOAD'] = ':'
import numpy as np
import robosuite as suite
from robosuite.utils.mjcf_utils import postprocess_model_xml

In [2]:
import os
import h5py
import argparse
import random

In [3]:
demo_path = '/home/ar4451/Data/RoboTurkPilot/bins-full/'
hdf5_path = "/home/ar4451/Data/RoboTurkPilot/bins-full/demo.hdf5"
f = h5py.File(hdf5_path, "r")
env_name = f["data"].attrs["env"]
demos = list(f["data"].keys())

In [4]:
env = suite.make(
        env_name,
        has_renderer=False,
        ignore_done=True,
        use_camera_obs=False,
        gripper_visualization=True,
        reward_shaping=True,
        control_freq=100,
    )

In [5]:
from stable_baselines3.ppo import PPO
import os
import time
from stable_baselines3.common.utils import set_random_seed

2023-05-11 03:10:47.672619: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 03:10:54.255474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 03:11:04.955200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
from robosuite.wrappers import GymWrapper

In [7]:
env2 = GymWrapper(
        suite.make(
            env_name,
            has_renderer=False,
            ignore_done=True,
            use_camera_obs=False,
            gripper_visualization=True,
            reward_shaping=True,
            control_freq=100,
        )
    )

for i_episode in range(20):
    observation = env2.reset()
    for t in range(500):
        # env.render()
        action = env2.action_space.sample()
        observation, reward, done, info = env2.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t + 1))
            break

adding key: robot-state
adding key: object-state


/opt/conda/anaconda/envs/robo/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [8]:
import robosuite as suite
import os
import yaml

from robosuite.wrappers import GymWrapper
from robosuite.environments.base import register_env

from stable_baselines3 import PPO
from stable_baselines3.common.save_util import save_to_zip_file, load_from_zip_file
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

import gym


def make_robosuite_env(env_id, rank, seed=0):
    def _init():
        tmp = suite.make(
            env_id,
            has_renderer=False,
            ignore_done=True,
            use_camera_obs=False,
            gripper_visualization=True,
            reward_shaping=True,
            control_freq=100,
        )
        env = GymWrapper(tmp)
        env = Monitor(env)
        return env
    set_random_seed(seed)
    return _init

In [ ]:
print(env_name)

# Default parameters
timesteps = 500000
nenv = 2  # number of parallel environments. This can speed up training when you have good CPUs
seed = 86
batch_size = 2048

# Set random seed
set_random_seed(seed)

env3 = SubprocVecEnv([make_robosuite_env(env_name, 0, seed)])

model = PPO("MlpPolicy", env3, verbose=1)
model.learn(total_timesteps=timesteps)

SawyerPickPlace


2023-05-11 03:12:56.036137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/anaconda/envs/robo/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 289  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 281         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013284478 |
|    clip_fraction        | 0.177       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.3       |
|    explained_variance   | 0.343       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0705     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0205     |
|    std                  | 0.998       |
|    value_loss           | 0.00419     |
-----------------

In [11]:
model.save("/home/ar4451/ppo.zip")

In [14]:
rewards = []
rewards2 = []
indirect_data = True
for i in range(1):
    ep = random.choice(demos)

    # read the model xml, using the metadata stored in the attribute for this episode
    model_file = f["data/{}".format(ep)].attrs["model_file"]
    model_path = os.path.join(demo_path, "models", model_file)
    with open(model_path, "r") as model_f:
        model_xml = model_f.read()

    env.reset()
    env2.reset()
    xml = postprocess_model_xml(model_xml)
    env.reset_from_xml_string(xml)
    env2.reset_from_xml_string(xml)
    env.sim.reset()
    env2.sim.reset()

    # load the flattened mujoco states
    states = f["data/{}/states".format(ep)][()]

    # load the initial state
    env.sim.set_state_from_flattened(states[0])
    env.sim.forward()
    env2.sim.set_state_from_flattened(states[0])
    env2.sim.forward()

    # load the actions and play them back open-loop
    jvels = f["data/{}/joint_velocities".format(ep)][()]
    grip_acts = f["data/{}/gripper_actuations".format(ep)][()]
    actions = np.concatenate([jvels, grip_acts], axis=1)
    num_actions = actions.shape[0]
    obs = env3.reset()
    ep_rewards = []
    ep_rewards2 = []
    if indirect_data:
        for j, action in enumerate(actions):
            _, reward1, done1, _  = env.step(action)
            predicted, _ = model.predict(obs)
            obs, reward2, done2, _  = env3.step(predicted)
            ep_rewards.append(reward1)
            ep_rewards2.append(reward2)
            if done1 or done2:
                break
    else:
        for state, action in zip(states, actions):
            env.sim.set_state_from_flattened(state)
            env.sim.forward()
            ep_rewards.append(env.reward(action))
    rewards.append(ep_rewards)
    rewards2.append(ep_rewards2)